![img](img/dark_library.png)

# **Memoria Técnica del Proyecto**

Steven Noboa, María Neches, Manuel Reina Águeda González

## 1. Introducción

* El presente documento proporciona una visión detallada del proyecto, incluyendo la implementación de una aplicación web basada en Flask para realizar consultas sobre documentos de texto utilizando la plataforma OpenAI. Asimismo, se describen los archivos y configuraciones asociadas, y se presentan las pruebas implementadas para garantizar el correcto funcionamiento del sistema.

## 2. Elaboración de un primer borrador del script y conexión a Langchain
+ Primer [template html](../app/templates/index.html) de la página principal
+ Desarrollo del script con una primera base de datos local de prueba.
+ Conexión a Langchain. En este paso, nos dimos cuenta de que las consultas de OpenAI que enviábamos desde Flask tenían un límite de 4096 tokens, por lo que no era capaz de leer archivos mayores de 4 kB. Por ello, decidimos fraccionar cada archivo para que lo enviara en porciones de 4096 tokens hasta que lo leyera en su totalidad. (Este procedimiento se puede ver a partir de la línea 78 del archivo [app_AWS.py](../app/app_AWS.py)). 

## 3. Creación de la instancia en AWS - RDS y de la base de datos MySQL
+ Creamos una instancia de la base de datos en la plataforma de AWS y nos conectamos a ella (puerto 3306).
+ Una vez conectados, creamos también la base de datos definitiva (gpt_database) y la tabla correspondiente (gpt_table), que constaba de las siguientes columnas:
```
gpt_table (
Registro DATETIME,
Nombre TEXT,
Archivo TEXT,
Consulta TEXT,
Respuesta TEXT)
```
+ Este proceso está desarrollado en el archivo [AWS_MySQL.ipynb](AWS_MySQL.ipynb)

- *Registro*: almacena la fecha y hora de la consulta realizada;
- *Nombre*: nombre de la persona que realiza la consulta;
- *Archivo*: nombre y extensión del archivo subido para la consulta;
- *Consulta*: prompt enviado a gpt para la consulta;
- *Respuesta*: resultados de la consulta a gpt.

## 4. Desarrollo de la App
+ Seguimos desarrollando la app y creando más endpoints. Inicialmente, el único endpoint existente (```/analizar_documento```) servía para introducir un nombre, un archivo y realizar una consulta a GPT sobre dicho archivo, y toda esta información se almacenaba en la base de datos (Este procedimiento se puede ver a partir de la línea 35 del archivo [app_AWS.py](../app/app_AWS.py)).
    + La app lee archivos con las extensiones ```.py```, ```.ipynb``` y ```.txt```.

+ Decidimos crear otros dos endpoints (```/consultas``` y ```/realizar_consulta```) que permiten también realizar consultas sobre los registros de la base de datos filtrándolos por nombre del usuario, aunque este se escriba en minúsculas o no esté completo (Este procedimiento se puede ver a partir de la línea 99 del archivo [app_AWS.py](../app/app_AWS.py)).

+ Para cada uno de estos endpoints, creamos una interfaz en html ([respuestas](../app/templates/respuestas.html), [consultas](../app/templates/consultas.html) y [results_BBDD](../app/templates/results_BBDD.html), respectivamente).

+ En el endpoint principal, habilitamos dos opciones para la API Key de OpenAI.
    + De forma predeterminada, la key es una variable de entorno y no está disponible para el usuario, de forma que cuando este realiza una consulta la key se introduce automáticamente de forma invisible.
    
    + La otra opción disponible es un campo que el usuario puede completar con su propia API Key para realizar consultas.

## 5. Test de la App
+ Desarrollamos los [test](../test/test.py) para poner la app a prueba y confirmar que nos devolvía código 200 en todos sus endpoints

![imagen](img/test_pasados.jpg)

## 6. Dockerización y despliegue de la App
+ Creamos los archivos de [Dockerfile](../app/dockerfile) y [requirements](../app/requirements.txt) para dockerizar la app.
+ Creamos la imagen final en docker (manuelreina/app_aws:v2) e hicimos un push para subirla a Docker Hub.

Para ejecutarla, se deberá ejecutar el siguiente comando:

```docker run -p 5000:5000 manuelreina/app_gpt:v1```

+ Los test están diseñados para poder ejecutarlos sobre la imagen dockerizada, aunque no se disponga del archivo local.